In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import vbll
from vbll.layers.classification import HetClassification

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "0"

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from dataclasses import dataclass

from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torchvision import transforms
from PIL import Image

import matplotlib.pyplot as pltw
from sklearn import metrics

from tqdm import tqdm
from matplotlib.pyplot import cm
import matplotlib.pyplot as plt 
%config InlineBackend.figure_format = 'retina'
floatX = torch.float32
RANDOM_SEED = 9927
rng = np.random.default_rng(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.random.manual_seed(RANDOM_SEED)
import torchvision
from torchvision.transforms import v2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
from bayesian_dpddm import DPDDM_Bayesian_Monitor, CNN_DPDDM_Model
import torch
torch.backends.cudnn.benchmark = True

In [4]:
train_transforms = v2.Compose([
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomCrop(size=[32,32], padding=4),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261]),
])
test_transforms = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261]),
])

In [5]:
cifar10train = torchvision.datasets.CIFAR10(root='data/', 
                                            transform=train_transforms,
                                            download=True)
cifar10test = torchvision.datasets.CIFAR10(root='data/', 
                                           train=False, 
                                           transform=test_transforms,
                                           download=True)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
class CIFAR101Dataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
    def __len__(self):
        return len(self.y)

In [7]:
with open('data/cifar10.1_v6_data.npy', 'rb') as f:
    ood_data = np.load(f)
with open('data/cifar10.1_v6_labels.npy', 'rb') as f:
    ood_labels = np.load(f)

In [8]:
norm_mean=[0.4914, 0.4822, 0.4465]
norm_std=[0.247, 0.243, 0.261]
transformed101data = torch.zeros(size=(len(ood_data), 3, 32, 32))
for idx in range(len(ood_data)):
    transformed101data[idx] = test_transforms(ood_data[idx])
transformed101labels = torch.as_tensor(ood_labels, dtype=torch.long)
dataset101 = CIFAR101Dataset(transformed101data, transformed101labels)

In [9]:
class train_cfg:
    NUM_EPOCHS = 2
    BATCH_SIZE = 64
    LR = 1e-3
    WD = 1e-4
    OPTIMIZER = torch.optim.AdamW
    CLIP_VAL = 1
    VAL_FREQ = 1
    VBLL = True
    
class cfg:
    IN_CHANNELS = 3
    MID_CHANNELS = 64
    OUT_FEATURES = 10
    KERNEL_SIZE = 3
    MID_LAYERS = 3
    POOL_DIMS = 2
    FLATTEN_DIM = 3136
    HIDDEN_DIM = 256
    DROPOUT = 0.2
    REG_WEIGHT = 1/len(cifar10train) * 10
    PARAM = 'diagonal'
    RETURN_OOD = False
    PRIOR_SCALE = 1
    WISHART_SCALE=1

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base_model = CNN_DPDDM_Model(cfg())
monitor = DPDDM_Bayesian_Monitor(
    model=base_model,
    trainset=cifar10train,
    valset=cifar10test,
    train_cfg=train_cfg(),
    device=device,
)

In [11]:
# Train base model
monitor.train_model(tqdm_enabled=True)

 50%|█████     | 1/2 [00:10<00:10, 10.48s/it]

Epoch:  0, train loss: 3.8117
Epoch:  0, valid loss: 1.3498


100%|██████████| 2/2 [00:20<00:00, 10.34s/it]


{'train_loss': [3.8116511201004846, 2.748416807950305],
 'val_loss': [1.3497511186417501, 1.1569332875263918],
 'train_acc': [0.3376358695652174, 0.47546355498721227],
 'val_acc': [0.5022890127388535, 0.5847929936305732],
 'ood_auroc': []}

In [12]:
monitor.pretrain_disagreement_distribution(
    cifar10test,
    Phi_size=10,
)

100%|██████████| 10/10 [00:01<00:00,  7.97it/s]


In [27]:
for k,v in {
    'cifar10-train': cifar10train,
    'cifar10-test': cifar10test,
    'cifar10.1': dataset101
}.items():
    tpr, _ = monitor.repeat_tests(v, n_repeats=100)
    print(f"{k}: {tpr}")

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:15<00:00,  6.37it/s]


cifar10-train: 0.88


100%|██████████| 100/100 [00:11<00:00,  8.86it/s]


cifar10-test: 0.12


100%|██████████| 100/100 [00:02<00:00, 33.94it/s]

cifar10.1: 1.0
